In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import re

def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

def scrap_mars():
    browser = init_browser()
    hemisphere =[]
    hemisphere_list=['Cerberus','Schiaparelli','Syrtis','Valles']
    for hemi in hemisphere_list:
        hemispheres={}
        url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
        browser.visit(url)
        html = browser.html
        soup = bs(html,'html.parser')
        browser.click_link_by_partial_text(hemi)
        html = browser.html
        soup = bs(html,'html.parser')
        hemispheres['image']=soup.find('a',target="_blank")['href']
        hemispheres['title']=soup.find('h2',class_="title").text
        hemisphere.append(hemispheres)
    browser.quit()
    
    browser = init_browser()

    # Visit mars
    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = bs(html, "html.parser")

    news_title = soup.findAll("div",{"class":"content_title"})[0].text.strip(' \t\n')
    news_p = soup.findAll("div",{"class":"rollover_description"})[0].text.strip(' \t\n')
    nasaData = {
        "news_title": news_title,
        "news_teaser": news_p
    }
    
    browser.quit()
    
    browser = init_browser()
    url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = bs(html, "html.parser")

    mars_weather = soup.find("div",{"class":"js-tweet-text-container"}).p.text
    mars_weather_clean = re.sub(r'(\s+|\n+|InSight|pic.twitter.com/J9DKptw2oo)', ' ', mars_weather)
    
    browser.quit()
    
    browser = init_browser()
    
    url_base = "https://www.jpl.nasa.gov"
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = bs(html, "html.parser")

    feature_mars = soup.find("div",{"class":"carousel_items"})
    relative_feature_image = feature_mars.a["data-fancybox-href"]
    feature_image = url_base + relative_feature_image
    feature_image_description = feature_mars.a["data-description"].strip('\r\t\n')
    feature_image_title = feature_mars.h1.text.strip('\r\t\n')

    marsData = {
        "feature_image": feature_image,
        "feature_image_title": feature_image_title,
        "feature_image_description": feature_image_description
    }
    
    browser.quit()
    
    dataPage = [{"hemisphere":hemisphere,"nasaData":nasaData,"weatherData":mars_weather_clean,"featuredImage":marsData}]
    return dataPage



In [2]:
scrap_mars()

[{'hemisphere': [{'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
    'title': 'Cerberus Hemisphere Enhanced'},
   {'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
    'title': 'Schiaparelli Hemisphere Enhanced'},
   {'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
    'title': 'Syrtis Major Hemisphere Enhanced'},
   {'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
    'title': 'Valles Marineris Hemisphere Enhanced'}],
  'nasaData': {'news_title': "NASA's Treasure Map for Water Ice on Mars",
   'news_teaser': 'A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.'},
  'weatherData': '  sol 375 (2019-12-16) low -97.5ºC (-143.6ºF) high -19.8ºC (-3.6ºF) winds from the SW at 5.6 m/s (12.5 mph) gustin

In [3]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as bs
import time
import re

url = "https://space-facts.com/mars/"
uClient = uReq(url)
html = uClient.read()
uClient.close()

soup = bs(html, "html.parser")

table_facts = soup.find("table",{"class":"tablepress"})
rows = table_facts.findAll("td")

data = []
for row in rows:
    name=row.text
    data.append(name)
    
title = [] 
value = [] 
for i in range(0, len(data)): 
    if i % 2: 
        value.append(data[i]) 
    else : 
        title.append(data[i]) 
dict_data = {"title":title,"value":value}
import pandas as pd
data_df = pd.DataFrame.from_dict(dict_data)
html_table=data_df.to_html(header=False, index=False, table_id=None)
html_table = re.sub(r'(\s+|\n)', ' ', html_table)
html_table

'<table border="1" class="dataframe"> <tbody> <tr> <td>Equatorial Diameter:</td> <td>6,792 km</td> </tr> <tr> <td>Polar Diameter:</td> <td>6,752 km</td> </tr> <tr> <td>Mass:</td> <td>6.39 × 10^23 kg (0.11 Earths)</td> </tr> <tr> <td>Moons:</td> <td>2 (Phobos &amp; Deimos)</td> </tr> <tr> <td>Orbit Distance:</td> <td>227,943,824 km (1.38 AU)</td> </tr> <tr> <td>Orbit Period:</td> <td>687 days (1.9 years)</td> </tr> <tr> <td>Surface Temperature:</td> <td>-87 to -5 °C</td> </tr> <tr> <td>First Record:</td> <td>2nd millennium BC</td> </tr> <tr> <td>Recorded By:</td> <td>Egyptian astronomers</td> </tr> </tbody> </table>'